# Topic modeling notebook

This notebook details the steps taken to clean the data and run topic modeling

In [28]:
import numpy as np
import pandas as pd
from gensim.utils import simple_preprocess
from lexical_diversity import lex_div as ld
import textstat
from nltk.corpus import stopwords
from nltk.stem.snowball import SwedishStemmer
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.model_selection import GridSearchCV
import pickle
from datetime import datetime

Import data

In [29]:
df_full = pd.read_csv("../dataset/lawline_data.csv")

Make new column with lowercase texts and remove all whitespace plus tabs/newlines

In [30]:
# Get only questions(::2)/answers(1::2)
df = df_full.iloc[::2, :]

In [31]:
df["text_clean"] = df["text"].str.lower()
df["text_clean"] = [" ".join(str(item).split()) for item in df["text_clean"]]

C:\Users\samue\AppData\Local\Temp\ipykernel_15420\2512645272.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = df["text"].str.lower()
C:\Users\samue\AppData\Local\Temp\ipykernel_15420\2512645272.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = [" ".join(str(item).split()) for item in df["text_clean"]]


Remove punctuation

In [32]:
df["text_clean"] = df["text_clean"].str.replace("[^\w\s]", "")

C:\Users\samue\AppData\Local\Temp\ipykernel_15420\1299325099.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text_clean"] = df["text_clean"].str.replace("[^\w\s]", "")
C:\Users\samue\AppData\Local\Temp\ipykernel_15420\1299325099.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = df["text_clean"].str.replace("[^\w\s]", "")


Remove numbers and replace more than one whitespace with single whitespace

In [33]:
df["text_clean"] = df["text_clean"].str.replace("[0-9]", "")
df["text_clean"] = df["text_clean"].str.replace("\s{2,}", " ")

C:\Users\samue\AppData\Local\Temp\ipykernel_15420\147325655.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text_clean"] = df["text_clean"].str.replace("[0-9]", "")
C:\Users\samue\AppData\Local\Temp\ipykernel_15420\147325655.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text_clean"] = df["text_clean"].str.replace("[0-9]", "")
C:\Users\samue\AppData\Local\Temp\ipykernel_15420\147325655.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["text_clean"] = df["text_clean"].str.replace("\s{2,}", " ")
C:\Users\samue\AppData\Local\Temp\ipykernel_15420\147325655.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [34]:
# Removing stopwords function
def remove_stopwords(texts, stop_words):
    return [
        [word for word in simple_preprocess(str(doc)) if word not in stop_words]
        for doc in texts
    ]

Turn texts to list of words and remove stop words. Then turn texts to term document frequency corpus.

In [35]:
# Turn texts to list
list_texts = df["text_clean"].values.tolist()

In [36]:
stop_words = stopwords.words("swedish")
stop_words.extend([
    "hej", "ska", "in", "vill", "alltså", "lawline", "även",
    "kommer", "fråga", "finns", 'gör', 'får', 'få', 'gäll', 'tack',
    'dock', 'kap', 'fall', 'säg', 'svar', 'rätt', 'enl', 'måst'
    ])

text_words = remove_stopwords(list_texts, stop_words)

Stem words

In [37]:
stemmer = SwedishStemmer()

text_stemmed = [[stemmer.stem(word) for word in doc] for doc in text_words]

In [38]:
docs_stemmed = [" ".join(doc) for doc in text_stemmed]

In [39]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(min_df=0.05, max_df=0.7,
                     stop_words=stop_words)

trunc_texts = cv.fit_transform(docs_stemmed)

Train LDA model on corpus with differing number of topics

In [40]:
# n_components = [3, 4, 5, 6, 7, 8, 9, 10, 12, 15, 20]
n_components = [6]
grid = {
    'doc_topic_prior': [.1, .5, 1, 5, 10],
    'topic_word_prior': [.1, .5, 1, 5, 10],
    'learning_decay': [.5, .7, 1]
}

lda_list = []
for n_topics in n_components:

    mod = LDA(
        max_iter=10,
        learning_method="online",
        random_state=42,
        batch_size=128,
        evaluate_every=-1,
        n_components=n_topics,
    )

    clf = GridSearchCV(mod, param_grid=grid, n_jobs=4, verbose=1)

    clf.fit(trunc_texts)

    lda_list.append(clf)

Fitting 5 folds for each of 75 candidates, totalling 375 fits


In [61]:
model = lda_list[2].best_estimator_

In [62]:
model.fit(trunc_texts)

LatentDirichletAllocation(doc_topic_prior=0.1, learning_method='online',
                          n_components=6, random_state=42, topic_word_prior=1)

In [40]:
def show_topics(vectorizer, lda_model, n_words=10):
    keywords = np.array(vectorizer.get_feature_names_out())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

In [63]:
topic_keywords = show_topics(vectorizer=cv, lda_model=model, n_words=10)

In [68]:
df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = [
    "Word " + str(i) for i in range(df_topic_keywords.shape[1])
]
df_topic_keywords.index = ["Topic " + str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords.transpose()

,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
Word 0,person,fast,avtal,barn,betal,lägen
Word 1,brott,sälj,lag,egendom,arbetsgiv,del
Word 2,beslut,köp,vänd,mak,år,se
Word 3,domstol,bil,innebär,bodelning,peng,använd
Word 4,dom,fel,avtalet,barnet,skuld,skad
Word 5,uppgift,huset,vän,del,tid,bost
Word 6,brottet,köpt,kräv,testament,kr,andr
Word 7,vänd,gåv,tid,arv,tillbak,ta
Word 8,polis,kräv,hälsning,ärv,dag,flytt
Word 9,andr,kunn,möj,enskild,månad,ekonomisk


Save model with pickle

In [3]:
model_name = '../models/lda_q_6.pkl'
with open(model_name, 'wb') as f:
    pickle.dump(model, f)

NameError: name 'model' is not defined

In [7]:
model_name = '../models/lda_q_6.pkl'
with open(model_name, 'rb') as f:
    model = pickle.load(f)

EOFError: Ran out of input

Get topic distribution, downsample to monthly average and save

In [1]:
topic_dists[0]

NameError: name 'topic_dists' is not defined

In [66]:
topic_dists = model.transform(trunc_texts)

In [88]:
df_topic_q = pd.DataFrame(topic_dists,
                          columns = [f'topic_{i}' for i in range(6)])

In [89]:
from datetime import datetime

df_topic_q['date'] = [datetime.strptime(date_str, "%d/%m/%Y") for date_str in df["date"].values]

In [90]:
df_topic_q.set_index('date', drop=True, inplace=True)

In [91]:
df_topic_q_down = df_topic_q.resample('M').mean()

In [93]:
df_topic_q_down.to_csv('../dataset/topic_q_downsampled.csv')